# Image Data Augmentation Exercise : **CIFAR-100**
---
## 데이터가 더 부족한 세상으로!

[여기 참고](https://www.cs.toronto.edu/~kriz/cifar.html)

## Keras Update

In [ ]:
!pip install keras-nightly

## Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets.cifar100 import load_data

In [ ]:
(train_x, train_y), (test_x, test_y) = load_data()
# (train_x, train_y), (test_x, test_y) = load_data(label_mode='coarse')

In [ ]:
np.unique(train_y)

In [ ]:
label_dict = {0:'apple', 1: 'aquarium_fish', 2: 'baby', 3: 'bear', 4: 'beaver', 5: 'bed', 6: 'bee', 7: 'beetle', 8: 'bicycle', 9: 'bottle',
              10: 'bowl', 11: 'boy',12: 'bridge',13: 'bus',14: 'butterfly',15: 'camel',16: 'can',17: 'castle',18: 'caterpillar',19: 'cattle',
              20: 'chair',21: 'chimpanzee',22: 'clock',23: 'cloud',24: 'cockroach',25: 'couch',26: 'cra',27: 'crocodile',28: 'cup',29: 'dinosaur',
              30: 'dolphin',31: 'elephant',32: 'flatfish',33: 'forest',34: 'fox',35: 'girl',36: 'hamster',37: 'house',38: 'kangaroo',39: 'keyboard',
              40: 'lamp',41: 'lawn_mower',42: 'leopard',43: 'lion',44: 'lizard',45: 'lobster',46: 'man',47: 'maple_tree',48: 'motorcycle',49: 'mountain',
              50: 'mouse',51: 'mushroom',52: 'oak_tree',53: 'orange',54: 'orchid',55: 'otter',56: 'palm_tree',57: 'pear',58: 'pickup_truck',59: 'pine_tree',
              60: 'plain',61: 'plate',62: 'poppy',63: 'porcupine',64: 'possum',65: 'rabbit',66: 'raccoon',67: 'ray',68: 'road',69: 'rocket',
              70: 'rose',71: 'sea',72: 'seal',73: 'shark',74: 'shrew',75: 'skunk',76: 'skyscraper',77: 'snail',78: 'snake',79: 'spider',
              80: 'squirrel',81: 'streetcar',82: 'sunflower',83: 'sweet_pepper',84: 'table',85: 'tank',86: 'telephone',87: 'television',88: 'tiger',89: 'tractor',
              90: 'train',91: 'trout',92: 'tulip',93: 'turtle',94: 'wardrobe',95: 'whale',96: 'willow_tree',97: 'wolf',98: 'woman',99: 'worm'
            }

label_dict[0]

* 데이터 살펴보기

In [ ]:
rand_i = np.random.randint(0, train_x.shape[0])

plt.title(f'idx: {rand_i} , class: { label_dict[train_y[rand_i][0]] }')
plt.imshow( train_x[rand_i] )
plt.show()

In [ ]:
rows = 5
fig, axes = plt.subplots(rows, len(label_dict), figsize=(len(label_dict), rows) )

for img_id in range(len(label_dict)) :
    imgs = train_x[train_y.reshape(-1)==img_id]
    imgs_len = len(imgs)

    for row_i in range(rows) :
        axe = axes[row_i, img_id]
        axe.imshow( imgs[np.random.randint(imgs_len)], interpolation='none' )
        axe.axis('off')

plt.tight_layout()
plt.show()

## Data Preprocessing

* Data split
    - training set : validation set = 8 : 2
    - 재현을 위한 난수 고정 : 2024

In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
train_x, val_x, train_y, val_y = tts(train_x, train_y, test_size=0.2, random_state=2024)

In [ ]:
train_x.shape, train_y.shape

* Scaling

    - min-max scaling (선택사항)
        1. RGB 정보 전체를 min-max
        2. R 따로 G 따로 B 따로 min-max, 그 후 하나로 통합

In [ ]:
# min-max scaling 1번 방법
max_n, min_n = train_x.max(), train_x.min()
max_n, min_n

In [ ]:
train_x_mm1 = (train_x - min_n) / (max_n - min_n)
val_x_mm1 = (val_x - min_n) / (max_n - min_n)
test_x_mm1 = (test_x - min_n) / (max_n - min_n)

In [ ]:
train_x_mm1.max(), train_x_mm1.min()

In [ ]:
# min-max scaling 2번 방법
tr_r_max, tr_r_min = train_x[:,:,:,0].max(), train_x[:,:,:,0].min()
tr_g_max, tr_g_min = train_x[:,:,:,1].max(), train_x[:,:,:,1].min()
tr_b_max, tr_b_min = train_x[:,:,:,2].max(), train_x[:,:,:,2].min()

In [ ]:
train_r_mm = (train_x[:,:,:,0] - tr_r_min) / (tr_r_max - tr_r_min)
train_g_mm = (train_x[:,:,:,1] - tr_g_min) / (tr_g_max - tr_g_min)
train_b_mm = (train_x[:,:,:,2] - tr_b_min) / (tr_b_max - tr_b_min)

In [ ]:
train_x_mm = np.stack((train_r_mm, train_g_mm, train_b_mm), axis=3)

In [ ]:
train_x_mm.shape

In [ ]:
val_r_mm = (val_x[:,:,:,0] - tr_r_min) / (tr_r_max - tr_r_min)
val_g_mm = (val_x[:,:,:,1] - tr_g_min) / (tr_g_max - tr_g_min)
val_b_mm = (val_x[:,:,:,2] - tr_b_min) / (tr_b_max - tr_b_min)

In [ ]:
val_x_mm = np.stack((val_r_mm, val_g_mm, val_b_mm), axis=3)

In [ ]:
val_x_mm.shape

In [ ]:
test_r_mm = (test_x[:,:,:,0] - tr_r_min) / (tr_r_max - tr_r_min)
test_g_mm = (test_x[:,:,:,1] - tr_g_min) / (tr_g_max - tr_g_min)
test_b_mm = (test_x[:,:,:,2] - tr_b_min) / (tr_b_max - tr_b_min)

In [ ]:
test_x_mm = np.stack((test_r_mm, test_g_mm, test_b_mm), axis=3)

In [ ]:
test_x_mm.shape

* One-hot encoding

In [ ]:
train_y.shape
train_y

In [ ]:
class_n = len(np.unique(train_y))

In [ ]:
from keras.utils import to_categorical

In [ ]:
train_y = to_categorical(train_y, class_n)
val_y = to_categorical(val_y, class_n)
test_y = to_categorical(test_y, class_n)

* Data shape 재확인

In [ ]:
train_x_mm.shape, train_y.shape

In [ ]:
train_y[0]

## Modeling : CNN

- 조건
    1. Sequential API, Functiona API 중 택일.
    2. Image Augmentation Layer를 최소 하나 이상 넣을 것! : [공식 문서 참고](https://keras.io/api/layers/preprocessing_layers/)
    3. [이 구조를 미니 버전으로 활용해봐도 좋다.](https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99DFA5415B38AC752E)
    4. DropOut, BatchNormalization 등의 기능도 같이 활용해보자.
    5. Early Stopping을 사용할 것.

In [ ]:
import keras

from keras.backend import clear_session
from keras.models import Model
from keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout, Conv2D, MaxPool2D
from keras.layers import RandomRotation, RandomTranslation, RandomZoom, RandomFlip

In [ ]:
# Functional API
# 1. 세션 클리어
clear_session()

# 2. 레이어 엮기
il = Input(shape=(32,32,3) )

al = RandomRotation(0.2)(il)
al = RandomTranslation(0.2, 0.2)(al)
al = RandomZoom(0.1)(al)
al = RandomFlip()(al)

hl = Conv2D(filters=64,        # 새롭게 제작하려는 feature map의 수! 서로 다른 filter의 수
            kernel_size=(3,3), # Conv filter의 가로세로 사이즈
            strides=(1,1),     # Conv filter의 이동 보폭
            padding='same',    # 1. 이전 feature map 사이즈 유지 | 2. 외곽 정보 더 반영
            activation='relu'  # 명시 주의!
            )(al)
hl = Conv2D(filters=64,        # 새롭게 제작하려는 feature map의 수! 서로 다른 filter의 수
            kernel_size=(3,3), # Conv filter의 가로세로 사이즈
            strides=(1,1),     # Conv filter의 이동 보폭
            padding='same',    # 1. 이전 feature map 사이즈 유지 | 2. 외곽 정보 더 반영
            activation='relu'  # 명시 주의!
            )(hl)
hl = MaxPool2D(pool_size=(2,2),# Pool filter 가로세로 사이즈
               strides=(2,2)   # Pool filter의 이동 보폭
               )(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.5)(hl)

hl = Conv2D(filters=128,        # 새롭게 제작하려는 feature map의 수! 서로 다른 filter의 수
            kernel_size=(3,3), # Conv filter의 가로세로 사이즈
            strides=(1,1),     # Conv filter의 이동 보폭
            padding='same',    # 1. 이전 feature map 사이즈 유지 | 2. 외곽 정보 더 반영
            activation='relu'  # 명시 주의!
            )(hl)
hl = Conv2D(filters=128,        # 새롭게 제작하려는 feature map의 수! 서로 다른 filter의 수
            kernel_size=(3,3), # Conv filter의 가로세로 사이즈
            strides=(1,1),     # Conv filter의 이동 보폭
            padding='same',    # 1. 이전 feature map 사이즈 유지 | 2. 외곽 정보 더 반영
            activation='relu'  # 명시 주의!
            )(hl)
hl = MaxPool2D(pool_size=(2,2),# Pool filter 가로세로 사이즈
               strides=(2,2)   # Pool filter의 이동 보폭
               )(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.5)(hl)

hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
ol = Dense(100, activation='softmax')(hl)

# 3. 모델 시작과 끝 지정
model = Model(il, ol)

# 4. 컴파일
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
model.summary()

* Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',       # 얼리스토핑 적용할 관측 대상
                   min_delta=0,              # 임계값.
                   patience=3,               # 성능 개선 미발생시 몇 epoch 더 진행할 것인가.
                   verbose=1,
                   restore_best_weights=True # 가장 성능 좋은 epoch의 가중치로 되돌림
                   )

* .fit( )

In [ ]:
model.fit(train_x_mm, train_y,
          validation_data=(val_x_mm, val_y),
          epochs=10000, verbose=1,
          callbacks=[es]                # 얼리스토핑 적용!
          )

* .evaluate( )

In [ ]:
model.evaluate(test_x_mm, test_y)

* .predict( )

In [ ]:
y_pred = model.predict(test_x_mm)

In [ ]:
# 원핫 인코딩 한 것을 다시 묶어주는 코드
# 평가 지표 및 실제 데이터 확인을 위해 필요

y_pred_arg = np.argmax(y_pred, axis=1)
test_y_arg = np.argmax(test_y, axis=1)

In [ ]:
y_pred_arg[0]

In [ ]:
test_y_arg[0]

* 평가 지표

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(test_y_arg, y_pred_arg)

In [ ]:
print( classification_report(test_y_arg, y_pred_arg, target_names=list(label_dict.values())) )

## Visualization

* 실제 데이터 확인

In [ ]:
rand_idx = np.random.randint(0, len(y_pred_arg))
test_idx = test_y_arg[rand_idx]
pred_idx = y_pred_arg[rand_idx]
class_prob = np.floor( y_pred[rand_idx]*100 )

print(f'idx = {rand_idx}')
print(f'해당 인덱스의 이미지는 {label_dict[test_idx]}')
print(f'모델의 예측 : {label_dict[pred_idx]}')
print(f'모델의 클래스별 확률 : ')
print('-------------------')
for idx, val in enumerate( list(label_dict.values()) ) :
    print(val, class_prob[idx])
print('=================================================')

if test_y_arg[rand_idx] == y_pred_arg[rand_idx] :
    print('정답')
else :
    print('땡')

plt.imshow(test_x[rand_idx])
plt.show()

* 틀린 이미지만 확인해보기

In [ ]:
temp = (test_y_arg == y_pred_arg)
false_idx = np.where(temp==False)[0]
false_len = len(false_idx)
false_len

In [ ]:
rand_idx = false_idx[np.random.randint(0, false_len)]
test_idx = test_y_arg[rand_idx]
pred_idx = y_pred_arg[rand_idx]
class_prob = np.floor( y_pred[rand_idx]*100 )

print(f'idx = {rand_idx}')
print(f'해당 인덱스의 이미지는 {label_dict[test_idx]}')
print(f'모델의 예측 : {label_dict[pred_idx]}')
print(f'모델의 클래스별 확률 : ')
print('-------------------')
for idx, val in enumerate( list(label_dict.values()) ) :
    print(val, class_prob[idx])
print('=================================================')

if test_y_arg[rand_idx] == y_pred_arg[rand_idx] :
    print('정답')
else :
    print('땡')

plt.imshow(test_x[rand_idx] )
plt.show()